In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

2.4.1


In [ ]:
# All general imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer 

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, Reshape, Conv2D, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Bidirectional, GlobalAveragePooling1D, GRU, GlobalMaxPooling1D, concatenate
from keras.optimizers import Adam
from keras.layers import LSTM, GRU, Conv1D, MaxPool1D, Activation, Add

from keras.models import Model, Sequential
from keras.layers.core import SpatialDropout1D

from keras.engine.topology import Layer
from keras.layers import Dense, Input, Embedding, Dropout, Activation, Conv1D, Softmax
from keras import initializers, regularizers, constraints, optimizers, layers
from keras import backend as K

from keras.callbacks import EarlyStopping

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
import io, os, gc

In [ ]:
#################### Importing FNC Datasets ####################
# Train set
train_df = pd.read_csv('FNC_Data/train_fnc_agdgonly.csv')
print(train_df.columns)
train_df.head()
le = LabelEncoder()
train_df['Stance'] = le.fit_transform(train_df['Stance'])
train_df.head()

# Test set
test_df = pd.read_csv('FNC_Data/test_fnc_agdgonly.csv')
print(test_df.columns)
test_df['Stance'] = le.transform(test_df['Stance'])
test_df.head()

Index(['Unnamed: 0', 'id', 'Body', 'Headline', 'Stance', 'Emotion_Labels',
       'Novelty_Quora', 'sentiment_scores_pre', 'sentiment_scores_hyp',
       'sentiment_pre_labels', 'sentiment_hyp_labels', 'Combined_Sentiment',
       'com_femotion', 'best_sentiments'],
      dtype='object')
Index(['Unnamed: 0', 'id', 'Body', 'Headline', 'Stance', 'Emotion_Labels',
       'Novelty_Quora', 'sentiment_scores_pre', 'sentiment_scores_hyp',
       'sentiment_pre_labels', 'sentiment_hyp_labels', 'Combined_Sentiment',
       'com_femotion', 'best_sentiments'],
      dtype='object')


,Unnamed: 0,id,Body,Headline,Stance,Emotion_Labels,Novelty_Quora,sentiment_scores_pre,sentiment_scores_hyp,sentiment_pre_labels,sentiment_hyp_labels,Combined_Sentiment,com_femotion,best_sentiments
0,0,1964,"Last week, Apple sent out the invites for its ...",EXCLUSIVE: Apple To Unveil The Long-Awaited Re...,0,1,1,0.088941,0.316288,1,1,0,1,0
1,1,1618,Did a woman claiming to have a third breast pl...,3-Boobed Woman a Fake,0,0,0,-0.143214,-0.140901,0,0,0,0,0
2,2,2316,A fourth grader named Aiden Steward was suspen...,Texas Boy Suspended For 'Threatening' Classmat...,0,0,1,0.010378,0.154774,1,1,0,1,0
3,3,1452,LOS ANGELES (CBS Seattle/AP) — A scorpion stun...,Woman stung by scorpion on Alaska Airlines flight,0,0,0,0.116980,-0.008654,1,0,1,0,1
4,4,1636,If the bizarre story about Joan Rivers' doctor...,Disgusting! Joan Rivers Doc Gwen Korovin’s Sic...,1,0,1,0.054843,-0.557444,1,0,1,0,1


In [ ]:
pre_bert_fnc = np.load("FNC_Data/pre_bert_fnc.npy")
hyp_bert_fnc = np.load("FNC_Data/hyp_bert_fnc.npy")
print('Premise', pre_bert_fnc.shape)
print('Hypothesis', hyp_bert_fnc.shape)

Premise (4518, 768)
Hypothesis (4518, 768)


In [ ]:
pre_bert_fnc_test = np.load("FNC_Data/pre_bert_test_fnc.npy")
hyp_bert_fnc_test = np.load("FNC_Data/hyp_bert_test_fnc.npy")
print('Premise', pre_bert_fnc_test.shape)
print('Hypothesis', hyp_bert_fnc_test.shape)

Premise (2600, 768)
Hypothesis (2600, 768)


In [ ]:
train_lst_1 = train_df['Body'].tolist()
print(len(train_lst_1))
train_lst_1[:5]
train_lst_2 = train_df['Headline'].tolist()
print(len(train_lst_2))
uq_tr_1 = list(set(train_lst_1))
uq_tr_2 = list(set(train_lst_2))
print(len(uq_tr_1))
print(len(uq_tr_2))
train_merged = uq_tr_1 + uq_tr_2
print('Train Length is', len(train_merged))
train_merged[:5]
test_lst_1 = test_df['Body'].tolist()
test_lst_2 = test_df['Headline'].tolist()
uq_ts_1 = list(set(test_lst_1))
uq_ts_2 = list(set(test_lst_2))
test_merged = uq_ts_1 + uq_ts_2
print('Test merged', len(test_merged))
total_dataset = train_merged + test_merged
print('Dataset length is', len(total_dataset))

4518
4518
842
958
Train Length is 1800
Test merged 961
Dataset length is 2761


In [ ]:
# Defining the tokenizer
def get_tokenizer(vocabulary_size):
  print('Training tokenizer...')
  tokenizer = Tokenizer(num_words= vocabulary_size)
  tweet_text = []
  print('Read {} Sentences'.format(len(total_dataset)))
  tokenizer.fit_on_texts(total_dataset)
  return tokenizer

In [ ]:
# For getting the embedding matrix
def get_embeddings():
  print('Generating embeddings matrix...')
  embeddings_file = '../resouces/glove.6B.300d.txt'
  embeddings_index = dict()
  with open(embeddings_file, 'r', encoding="utf-8") as infile:
    for line in infile:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      embeddings_index[word] = vector
	# create a weight matrix for words in training docs
  vocabulary_size = len(embeddings_index)
  embeddinds_size = list(embeddings_index.values())[0].shape[0]
  print('Vocabulary = {}, embeddings = {}'.format(vocabulary_size, embeddinds_size))
  tokenizer = get_tokenizer(vocabulary_size)
  embedding_matrix = np.zeros((vocabulary_size, embeddinds_size))
  considered = 0
  total = len(tokenizer.word_index.items())
  for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
      print(word, index)
      continue
    else:
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
        considered += 1
  print('Considered ', considered, 'Left ', total - considered)			
  return embedding_matrix, tokenizer, embeddings_index

In [ ]:
def get_data(tokenizer, MAX_LENGTH, input_df):
  print('Loading data')
  X1, X2, Y = [], [], []
	# with open(input_file) as infile:
	# 	for line in infile:
	# 		data = line.split(',')
	# 		text, annotation = data[2], data[1]
			
	# 		if annotation == "MET":
	# 			X.append(text)
	# 			Y.append("1")
	# 		elif annotation == "Non_MET" or annotation == "Help":	
	# 			X.append(text)
	# 			Y.append("0")
  X1 = input_df['Body'].tolist()
  X2 = input_df['Headline'].tolist()
  Y = input_df['Stance'].tolist()
  Y_nv = input_df["Novelty_Quora"].values
  Y_em = input_df["com_femotion"].values
  #Y_st = input_df['best_sentiments'].tolist()
  Y_st = input_df['Combined_Sentiment'].tolist()
  
  assert len(X1) == len(X2) == len(Y)
  sequences_1 = tokenizer.texts_to_sequences(X1)
  sequences_2 = tokenizer.texts_to_sequences(X2)
	# for i, s in enumerate(sequences):
	# 	sequences[i] = sequences[i][-250:]
  X1 = pad_sequences(sequences_1, maxlen=MAX_LENGTH)
  X2 = pad_sequences(sequences_2, maxlen=MAX_LENGTH)
  Y_fnc = np.array(Y)
  Y_nv = np.array(Y_nv)
  Y_em = np.array(Y_em)
  Y_st = np.array(Y_st)

  return X1, X2, Y_fnc, Y_nv, Y_em, Y_st

In [ ]:
embedding_matrix, tokenizer, embeddings_index = get_embeddings()

Generating embeddings matrix...
Vocabulary = 400000, embeddings = 300
Training tokenizer...
Read 2761 Sentences
Considered  20628 Left  7234


In [ ]:
MAX_LENGTH = 100
# read ml data
X1, X2, Y_fnc, Y_nv, Y_em, Y_st = get_data(tokenizer, MAX_LENGTH, train_df)

Loading data


In [ ]:
X1_test, X2_test, Y_fnc_test, Y_nv_test, Y_em_test, Y_st_test = get_data(tokenizer, MAX_LENGTH, test_df)

Loading data


In [ ]:
# Scaffold labels
novel = embeddings_index['original']
duplicate = embeddings_index['duplicate']
emotion_true = embeddings_index['anticipation']+embeddings_index['sadness']+embeddings_index['joy']+embeddings_index['trust']
emotion_false = embeddings_index['anger']+embeddings_index['fear']+embeddings_index['disgust']+embeddings_index['surprise']
sentiment_pos = embeddings_index['positive']
sentiment_neg = embeddings_index['negative']

In [ ]:
# Novelty Bias
true_train_labels = train_df['Stance'].tolist()
true_test_labels = test_df['Stance'].tolist()
train_bias_nv = []
test_bias_nv = []
zero_vector = np.zeros((300,))
for i, row in train_df.iterrows():
    if row['Novelty_Quora'] == 0 and row['Stance'] == 1:
        train_bias_nv.append(novel)
    elif row['Novelty_Quora'] == 1 and row['Stance'] == 0:
        train_bias_nv.append(duplicate)
    else:
        train_bias_nv.append(zero_vector)
for i, row in test_df.iterrows():
    if row['Novelty_Quora'] == 0 and row['Stance'] == 1:
        test_bias_nv.append(novel)
    elif row['Novelty_Quora'] == 1 and row['Stance'] == 0:
        test_bias_nv.append(duplicate)
    else:
        test_bias_nv.append(zero_vector)
        #print('Error in Test please check')
train_bias_nv = np.stack(train_bias_nv)
test_bias_nv = np.stack(test_bias_nv)
print('Train bias', train_bias_nv.shape)
print('Test bias', test_bias_nv.shape)

Train bias (4518, 300)
Test bias (2600, 300)


In [ ]:
# Emotion Bias
train_bias_em = []
test_bias_em = []
zero_vector = np.zeros((300,))
for i in range(len(train_df)):
    pre = train_df.loc[i, 'com_femotion']
    if train_df.loc[i, 'Stance'] == 0 and pre == 1:
        train_bias_em.append(emotion_true)
    elif train_df.loc[i, 'Stance'] == 1 and pre == 0:
        train_bias_em.append(emotion_false)
    else:
        train_bias_em.append(zero_vector)
for i in range(len(test_df)):
    pre = test_df.loc[i, 'com_femotion']
    if test_df.loc[i, 'Stance'] == 0 and pre == 1:
        test_bias_em.append(emotion_true)
    elif test_df.loc[i, 'Stance'] == 1 and pre == 0:
        test_bias_em.append(emotion_false)
    else:
        test_bias_em.append(zero_vector)
train_bias_em = np.stack(train_bias_em)
test_bias_em = np.stack(test_bias_em)
print('Train bias', train_bias_em.shape)
print('Test bias', test_bias_em.shape)

Train bias (4518, 300)
Test bias (2600, 300)


In [ ]:
# Sentiment Bias
train_bias_st = []
test_bias_st = []
zero_vector = np.zeros((300,))
for i in range(len(train_df)):
    #hyp = train_df.loc[i, 'best_sentiments']
    hyp = train_df.loc[i, 'Combined_Sentiment']
    if hyp == 0 and train_df.loc[i, 'Stance'] == 0:
        train_bias_st.append(sentiment_pos)
    # elif hyp == 0 and train_df.loc[i, 'stance'] == 2:
    #     train_bias_em.append(emotion_true)
    elif hyp == 1 and train_df.loc[i, 'Stance'] == 1:
        train_bias_st.append(sentiment_neg)
    else:
        #print('in here')
        train_bias_st.append(zero_vector)
for i in range(len(test_df)):
    #hyp = test_df.loc[i, 'best_sentiments']
    hyp = test_df.loc[i, 'Combined_Sentiment']
    if hyp == 0 and test_df.loc[i, 'Stance'] == 0:
        test_bias_st.append(sentiment_pos)
    # elif hyp == 0 and test_df.loc[i, 'stance'] == 2:
    #     test_bias_em.append(emotion_true)
    elif hyp == 1 and test_df.loc[i, 'Stance'] == 1:
        test_bias_st.append(sentiment_neg)
    else:
        test_bias_st.append(zero_vector)
train_bias_st = np.stack(train_bias_st)
test_bias_st = np.stack(test_bias_st)
print('Train bias', train_bias_st.shape)
print('Test bias', test_bias_st.shape)

Train bias (4518, 300)
Test bias (2600, 300)


In [ ]:
# Considering the final train and test bias
# train_bias = np.add(train_bias_nv, train_bias_st)
# test_bias = np.add(test_bias_nv, test_bias_st)
train_bias = np.add(train_bias_nv, train_bias_em, train_bias_st)
test_bias = np.add(test_bias_nv, test_bias_em, test_bias_st)

In [ ]:
# Creating one-hot encodings
y_train_nv = keras.utils.to_categorical(Y_nv)
print(y_train_nv)
y_train_em = keras.utils.to_categorical(Y_em)
print(y_train_em)
y_train_fnc = keras.utils.to_categorical(Y_fnc)
print(y_train_fnc)
y_train_st = keras.utils.to_categorical(Y_st)
print(y_train_st)
y_test_nv = keras.utils.to_categorical(Y_nv_test)
print(y_test_nv)
y_test_em = keras.utils.to_categorical(Y_em_test)
print(y_test_em)
y_test_fnc = keras.utils.to_categorical(Y_fnc_test)
print(y_test_fnc)
y_test_st = keras.utils.to_categorical(Y_st_test)
print(y_test_st)

[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]
[[1. 0.]
 [1. 0.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]
[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]
[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [ ]:
from sklearn.model_selection import train_test_split
VALIDATION_RATIO = 0.1
RANDOM_STATE = 9527
x1_train, x1_val, \
x2_train, x2_val, \
x1_train_bert, x1_val_bert, \
x2_train_bert, x2_val_bert, \
y_train_nv, y_val_nv, \
y_train_em, y_val_em, \
y_train_st, y_val_st, \
y_train_fnc, y_val_fnc, \
train_bias, val_bias, \
train_bias_nv, val_bias_nv, \
train_bias_em, val_bias_em, \
train_bias_st, val_bias_st = \
    train_test_split(
        X1, X2, 
        pre_bert_fnc, hyp_bert_fnc,
        y_train_nv, y_train_em, y_train_st,
        y_train_fnc, train_bias,
        train_bias_nv, train_bias_em, train_bias_st,
        test_size=VALIDATION_RATIO, 
        random_state=RANDOM_STATE
)

In [ ]:
print("Training Set")
print("-" * 10)
print(f"x1_train: {x1_train.shape}")
print(f"x2_train: {x2_train.shape}")
print(f"y_train_cs : {y_train_fnc.shape}")
print(f"Train_Bias : {train_bias.shape}")

print("-" * 10)
print(f"x1_val:   {x1_val.shape}")
print(f"x2_val:   {x2_val.shape}")
print(f"y_val_cs :   {y_val_fnc.shape}")
print(f"Val_Bias : {val_bias.shape}")
print("-" * 10)
print("Test Set")

Training Set
----------
x1_train: (4066, 100)
x2_train: (4066, 100)
y_train_cs : (4066, 2)
Train_Bias : (4066, 300)
----------
x1_val:   (452, 100)
x2_val:   (452, 100)
y_val_cs :   (452, 2)
Val_Bias : (452, 300)
----------
Test Set


In [ ]:
NUM_CLASSES = 2

MAX_SEQUENCE_LENGTH = 100

NUM_LSTM_UNITS = 150

MAX_NUM_WORDS = embedding_matrix.shape[0]

NUM_EMBEDDING_DIM = embedding_matrix.shape[1]

In [ ]:
# BERT + Normal Grand Model

NUM_LSTM_UNITS = 150

top_input_wd = Input(
    shape=(MAX_SEQUENCE_LENGTH, ), 
    dtype='int32')
bm_input_wd = Input(
    shape=(MAX_SEQUENCE_LENGTH, ), 
    dtype='int32')

embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM, weights = [embedding_matrix], trainable = True)
top_embedded_wd = embedding_layer(
    top_input_wd)
bm_embedded_wd = embedding_layer(
    bm_input_wd)

source_lstm_wd = Bidirectional(LSTM(NUM_LSTM_UNITS, return_sequences=True, recurrent_dropout = 0.3))
shared_lstm_wd = Bidirectional(LSTM(NUM_LSTM_UNITS, activation='tanh', recurrent_dropout = 0.3))
top_source_wd = source_lstm_wd(top_embedded_wd)
bm_source_wd = source_lstm_wd(bm_embedded_wd)

source_comb_wd = concatenate(
    [top_source_wd, bm_source_wd],
    axis=-1
    )
lstm_ops_wd = shared_lstm_wd(source_comb_wd)   # 300D vector


top_input_bt = Input(
    shape=(768, ), 
    dtype='float32')
bm_input_bt = Input(
    shape=(768, ), 
    dtype='float32')
bias_input = Input(
    shape = (300, ),
    dtype = 'float32')
bias_input_nv = Input(
    shape = (300, ),
    dtype = 'float32')
bias_input_em = Input(
    shape = (300, ),
    dtype = 'float32')
bias_input_st = Input(
    shape = (300, ),
    dtype = 'float32')


top_embedded_bt = Reshape((1, 768, ))(top_input_bt)
bm_embedded_bt = Reshape((1, 768, ))(bm_input_bt)

source_lstm_bt = Bidirectional(LSTM(NUM_LSTM_UNITS, return_sequences=True, recurrent_dropout = 0.3))
shared_lstm_bt = Bidirectional(LSTM(NUM_LSTM_UNITS, activation='tanh', recurrent_dropout = 0.3))
top_source_bt = source_lstm_bt(top_embedded_bt)
bm_source_bt = source_lstm_bt(bm_embedded_bt)

source_comb_bt = concatenate(
    [top_source_bt, bm_source_bt],
    axis=-1
    )
lstm_ops_bt = shared_lstm_bt(source_comb_bt)  #300D vector

#merged = Add()([top_output, bm_output])
#merged_bd = Add()([lstm_ops, bias_input])

# Bert and Normal Combination
comb_features = lstm_ops_wd + lstm_ops_bt

comb_features_nv = concatenate(
    [lstm_ops_wd+lstm_ops_bt+bias_input_nv, lstm_ops_wd*lstm_ops_bt*bias_input_nv],
    axis=-1
    )
comb_features_em = concatenate(
    [lstm_ops_wd+lstm_ops_bt+bias_input_em, lstm_ops_wd*lstm_ops_bt*bias_input_em],
    axis=-1
    )
comb_features_st = concatenate(
    [lstm_ops_wd+lstm_ops_bt+bias_input_st, lstm_ops_wd*lstm_ops_bt*bias_input_st],
    axis=-1
    )

comb_features_fnc = concatenate(
    [lstm_ops_wd+lstm_ops_bt+bias_input, lstm_ops_wd*lstm_ops_bt*bias_input],
    axis=-1
    )

pre_nv = Dense(
    units=64, 
    activation='tanh',
    name = 'pre_nv')(comb_features_nv)

pre_em = Dense(
    units=64, 
    activation='tanh',
    name = 'pre_em')(comb_features_em)

pre_st = Dense(
    units=64, 
    activation='tanh',
    name = 'pre_st')(comb_features_st)

pre_fnc = Dense(
    units=64, 
    activation='tanh',
    name = 'pre_fnc')(comb_features_fnc)

dense_nv =  Dense(
    units=NUM_CLASSES, 
    activation='softmax',
    name = 'nv')

dense_em =  Dense(
    units=NUM_CLASSES, 
    activation='softmax',
    name = 'em')

dense_st =  Dense(
    units=NUM_CLASSES, 
    activation='softmax',
    name = 'st')

dense_fnc =  Dense(
    units=NUM_CLASSES, 
    activation='softmax',
    name = 'fnc')

predictions_nv = dense_nv(pre_nv)
predictions_em = dense_em(pre_em)
predictions_st = dense_st(pre_st)
predictions_fnc = dense_fnc(pre_fnc)

model = Model(
    inputs=[top_input_wd, bm_input_wd, top_input_bt, bm_input_bt, bias_input, bias_input_nv, bias_input_em, bias_input_st], 
    outputs=[predictions_nv, predictions_em, predictions_st, predictions_fnc])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 768)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 768)]        0                                            
______________________________________________________________________________________________

In [ ]:
from keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
lr = 1e-3
opt = Adam(lr=lr, decay=lr/50)
model.compile(
    optimizer='adam',
    loss={'nv':'categorical_crossentropy', 'em':'categorical_crossentropy', 'st':'categorical_crossentropy', 'fnc':'categorical_crossentropy'},
    loss_weights={'nv': 1, 'em':1, 'st':1, 'fnc': 1},
    metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='multitask_fncbias.h5', verbose=1, save_best_only=True)

In [ ]:
# MultiTask BERT Model
BATCH_SIZE = 256
NUM_EPOCHS = 50
stop = [EarlyStopping(monitor='val_loss', patience=0.001)]
history = model.fit(x=[x1_train, x2_train, x1_train_bert, x2_train_bert, train_bias, train_bias_nv, train_bias_em, train_bias_st],
                    y=[y_train_nv, y_train_em, y_train_st, y_train_fnc],
                    batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    validation_data=(
                      [x1_val, x2_val, x1_val_bert, x2_val_bert, val_bias, val_bias_nv, val_bias_em, val_bias_st], 
                      [y_val_nv, y_val_em, y_val_st, y_val_fnc]
                    ),
                    shuffle=True,
                    callbacks=[stop, checkpointer],
          )

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report
predictions = model.predict(
    [X1_test, X2_test, pre_bert_fnc_test, hyp_bert_fnc_test, test_bias, test_bias_nv, test_bias_em, test_bias_st])
# print('Accuracy is')
# print(metrics.accuracy_score(y_test, y_pred, sample_weight = reduced_test_weights)*100)
# print(classification_report(y_test, y_pred, target_names = ['agreed', 'disagreed'], sample_weight = reduced_test_weights))

In [ ]:
print(np.stack(predictions).shape)
print(predictions[0].shape)
print(predictions[1].shape)
print(predictions[2].shape)
print(predictions[3].shape)

(4, 2600, 2)
(2600, 2)
(2600, 2)
(2600, 2)
(2600, 2)


In [ ]:
# Result Labels
res_df = pd.DataFrame()

In [ ]:
y_pred = [idx for idx in np.argmax(predictions[3], axis=1)]
res_df['Fake_News_Labels'] = y_pred
#print(y_pred)
print('FNC Accuracy is')
print(metrics.accuracy_score(Y_fnc_test, y_pred)*100)
print(classification_report(Y_fnc_test, y_pred, target_names = ['agree', 'disagree']))

In [ ]:
y_pred = [idx for idx in np.argmax(predictions[2], axis=1)]
res_df['Sentiment_Labels'] = y_pred
print('Sentiment Accuracy is')
print(metrics.accuracy_score(Y_st_test, y_pred)*100)
print(classification_report(Y_st_test, y_pred, target_names = ['same', 'different']))

In [ ]:
y_pred = [idx for idx in np.argmax(predictions[1], axis=1)]
res_df['Emotion_Labels'] = y_pred
print('Emotion Accuracy is')
print(metrics.accuracy_score(Y_em_test, y_pred)*100)
print(classification_report(Y_em_test, y_pred, target_names = ['true', 'false']))

In [ ]:
y_pred = [idx for idx in np.argmax(predictions[0], axis=1)]
res_df['Novelty_Labels'] = y_pred
print('NV Accuracy is')
print(metrics.accuracy_score(Y_nv_test, y_pred)*100)
print(classification_report(Y_nv_test, y_pred, target_names = ['novel', 'duplicate']))

In [ ]:
# Saving the labels
res_df.to_csv("FNC_MtaskRes.csv", index = False)